In [36]:
import pandas as pd
import numpy as np
from collections import Counter
#sklearn
from sklearn import preprocessing
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.feature_extraction.text import CountVectorizer

#nltk
from nltk.corpus import stopwords

In [2]:
df = pd.read_csv("../data/final_dataset.csv")

In [3]:
df.head()

,id,description,keywords,collection,production_countries,popularity,average_vote,num_votes,language,imdb_id,...,date,title,director,production_companies,cast,revenue,budget,description2,tagline,movie_classification
0,2,Taisto Kasurinen is a Finnish coal miner whose...,"underdog, prison, factory worker, prisoner, he...",NaN,Finland,3.860491,7.1,44.0,fi,tt0094675,...,1988-10-21,Ariel,Aki Kaurismäki,"Villealfa Filmproduction Oy, Finnish Film Foun...","Turo Pajala, Susanna Haavisto, Matti Pellonpää...",0.0,0,A Finnish man goes to the city to find a job a...,NaN,unclassified
1,3,"An episode in the life of Nikander, a garbage ...","salesclerk, helsinki, garbage, independent film",NaN,Finland,2.292110,7.1,35.0,fi,tt0092149,...,1986-10-16,Shadows in Paradise,Aki Kaurismäki,Villealfa Filmproduction Oy,"Matti Pellonpää, Kati Outinen, Sakari Kuosmane...",0.0,0,"An episode in the life of Nikander, a garbage ...",NaN,unclassified
2,5,It's Ted the Bellhop's first night on the job....,"hotel, new year's eve, witch, bet, hotel room,...",NaN,United States of America,9.026586,6.5,539.0,en,tt0113101,...,1995-12-22,Four Rooms,"Allison Anders, Alexandre Rockwell, Robert Rod...","Miramax Films, A Band Apart","Tim Roth, Antonio Banderas, Jennifer Beals, Ma...",4257354.0,4000000,Four interlocking tales that take place in a f...,Twelve outrageous guests. Four scandalous requ...,minor success
3,6,"While racing to a boxing match, Frank, Mike, J...","chicago, drug dealer, boxing match, escape, on...",NaN,"Japan, United States of America",5.538671,6.4,79.0,en,tt0107286,...,1993-10-15,Judgment Night,Stephen Hopkins,"Universal Pictures, Largo Entertainment, JVC E...","Emilio Estevez, Cuba Gooding Jr., Denis Leary,...",12526677.0,0,Four friends on their way to a boxing match ge...,Don't move. Don't whisper. Don't even breathe.,unclassified
4,11,Princess Leia is captured and held hostage by ...,"android, galaxy, hermit, death star, lightsabe...",Star Wars Collection,United States of America,42.149697,8.1,6778.0,en,tt0076759,...,1977-05-25,Star Wars: Episode IV - A New Hope,George Lucas,"Lucasfilm, Twentieth Century Fox Film Corporation","Mark Hamill, Harrison Ford, Carrie Fisher, Pet...",775512064.0,11000000,Luke Skywalker joins forces with a Jedi Knight...,"A long time ago in a galaxy far, far away...",super hit


In [4]:
df.isnull().sum()

id                          0
description               952
keywords                14325
collection              40926
production_countries     6269
popularity                  0
average_vote                0
num_votes                   0
language                   11
imdb_id                     0
poster_url                377
runtime                    15
date                       72
title                       0
director                  119
production_companies     9791
cast                      652
revenue                     0
budget                      0
description2              709
tagline                 25011
movie_classification        0
dtype: int64

In [5]:
df.drop(columns=["id","collection","poster_url","date","director","cast","imdb_id","production_companies"],inplace=True)

In [6]:
df[["description","description2"]].fillna(' ', inplace=True)

/home/alo-reza/.pyenv/versions/3.6.6/envs/pygpu/lib/python3.6/site-packages/pandas/core/frame.py:4259: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  **kwargs


In [7]:
df["new description"] = df.apply(lambda x: x["description"] if str(x["description"])==str(x["description2"]) 
                                 else str(x["description"])+" "+ str(x["description2"]),axis=1)

In [8]:
df["text data"] = df["title"]+" "+df["keywords"]+" "+df["tagline"]+" "+df["new description"]

In [9]:
df.columns

Index(['description', 'keywords', 'production_countries', 'popularity',
       'average_vote', 'num_votes', 'language', 'runtime', 'title', 'revenue',
       'budget', 'description2', 'tagline', 'movie_classification',
       'new description', 'text data'],
      dtype='object')

In [10]:
col = ['popularity','average_vote','num_votes','runtime'
       ,'revenue', 'budget', 'movie_classification','text data']

In [11]:
df = df[col]

In [12]:
df.shape

(45408, 8)

In [13]:
df.isnull().sum()

popularity                  0
average_vote                0
num_votes                   0
runtime                    15
revenue                     0
budget                      0
movie_classification        0
text data               28267
dtype: int64

In [14]:
df["text data"].fillna(" ",inplace=True)

In [15]:
df["runtime"].fillna(0,inplace = True)

In [16]:
df.isnull().sum()

popularity              0
average_vote            0
num_votes               0
runtime                 0
revenue                 0
budget                  0
movie_classification    0
text data               0
dtype: int64

In [17]:
df.head()

,popularity,average_vote,num_votes,runtime,revenue,budget,movie_classification,text data
0,3.860491,7.1,44.0,72.0,0.0,0,unclassified,
1,2.292110,7.1,35.0,74.0,0.0,0,unclassified,
2,9.026586,6.5,539.0,98.0,4257354.0,4000000,minor success,"Four Rooms hotel, new year's eve, witch, bet, ..."
3,5.538671,6.4,79.0,110.0,12526677.0,0,unclassified,"Judgment Night chicago, drug dealer, boxing ma..."
4,42.149697,8.1,6778.0,121.0,775512064.0,11000000,super hit,"Star Wars: Episode IV - A New Hope android, ga..."


In [18]:
df.columns

Index(['popularity', 'average_vote', 'num_votes', 'runtime', 'revenue',
       'budget', 'movie_classification', 'text data'],
      dtype='object')

In [19]:
minmax = preprocessing.MinMaxScaler()
col = ['popularity', 'average_vote', 'num_votes', 'runtime', 'revenue','budget']
df[col] = minmax.fit_transform(df[col])

In [20]:
stop_words = stopwords.words("english")
vectorizer = CountVectorizer(stop_words=stop_words)
model = vectorizer.fit(df['text data'])
docs = vectorizer.transform(df['text data'])
lda = LatentDirichletAllocation(10)
lda.fit(docs)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=10, n_jobs=None,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=0)

In [21]:
lda_df = pd.DataFrame(lda.transform(docs))

In [22]:
lda_df.describe()

,0,1,2,3,4,5,6,7,8,9
count,45408.000000,45408.000000,45408.000000,45408.000000,45408.000000,45408.000000,45408.000000,45408.000000,45408.000000,45408.000000
mean,0.084582,0.149626,0.088495,0.091373,0.112268,0.087337,0.099602,0.090799,0.088662,0.107255
std,0.095690,0.187337,0.103091,0.111782,0.140838,0.104004,0.119488,0.110425,0.104729,0.135504
min,0.000118,0.000273,0.000279,0.000118,0.000118,0.000118,0.000118,0.000118,0.000118,0.000265
25%,0.002703,0.100000,0.002858,0.002858,0.058628,0.002778,0.003846,0.002858,0.002858,0.004349
50%,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
75%,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
max,0.993705,0.995889,0.996295,0.994266,0.992621,0.995566,0.992967,0.996511,0.993282,0.994577


In [23]:
df.shape,lda_df.shape

((45408, 8), (45408, 10))

In [24]:
df = pd.concat([df,lda_df],axis=1)

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [26]:
X = df.drop(columns=["text data","movie_classification"])
y = df["movie_classification"]
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0,stratify = y)

In [27]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)

/home/alo-reza/.pyenv/versions/3.6.6/envs/pygpu/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [28]:
clf.score(X_test,y_test)

0.8683051444679352

In [29]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((34056, 16), (34056,), (11352, 16), (11352,))

In [32]:
y_train.value_counts()

unclassified       28427
minor success       1365
box office bomb     1277
super hit           1078
flop                1024
blockbuster          885
Name: movie_classification, dtype: int64

In [33]:
y_pred = clf.predict(X_test)

In [37]:
Counter(y_pred)

Counter({'unclassified': 10559,
         'super hit': 176,
         'minor success': 503,
         'box office bomb': 39,
         'blockbuster': 50,
         'flop': 25})

In [38]:
y_test.value_counts()

unclassified       9476
minor success       455
box office bomb     426
super hit           359
flop                341
blockbuster         295
Name: movie_classification, dtype: int64

In [51]:
from sklearn.metrics import multilabel_confusion_matrix,recall_score,precision_score

In [52]:
precision_score(y_test,y_pred,average="weighted")

0.8367403982661797

In [50]:
recall_score(y_test,y_pred,average="weighted")

0.8683051444679352